# Debug Date Parsing Issues

This notebook investigates why date values are showing up as NaT in the review data.

In [1]:
# Import required libraries
import os
import sys
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to Python path")

# Load environment variables
load_dotenv()

Added /Users/dipesh/Local-Projects/indigo-reviews-ai to Python path


True

In [ ]:
# Load the CSV file directly with a sample for testing
csv_path = os.path.join(project_root, 'data', 'reviews.csv')
if os.path.exists(csv_path):
    print(f"Loading reviews from: {csv_path}")
    # Load only a sample of 1000 reviews to speed up testing
    raw_df = pd.read_csv(csv_path, low_memory=False, nrows=1000)
    print(f"Successfully loaded {len(raw_df)} sample reviews from CSV file")
else:
    print(f"CSV file not found at {csv_path}")

In [ ]:
# Check the columns and their types
print("DataFrame Info:")
raw_df.info()

# Focus on date-related columns
date_cols = [col for col in raw_df.columns if 'date' in col.lower() or 'time' in col.lower()]
print(f"\nPotential date columns: {date_cols}")

In [ ]:
# Examine the date column
if 'date' in raw_df.columns:
    print("Sample values from 'date' column:")
    print(raw_df['date'].sample(10).tolist())
    
    # Check unique values
    unique_dates = raw_df['date'].nunique()
    print(f"\nNumber of unique date values: {unique_dates}")
    
    # Check for nulls
    null_dates = raw_df['date'].isna().sum()
    print(f"Number of null date values: {null_dates} ({null_dates/len(raw_df)*100:.2f}%)")
    
    # Try to determine the date format if it's a string
    if pd.api.types.is_object_dtype(raw_df['date']):
        non_null_dates = raw_df['date'].dropna()
        if len(non_null_dates) > 0:
            print(f"\nFirst few non-null date values:")
            print(non_null_dates.head(5).tolist())

In [ ]:
# Check if timestamp column exists and might be useful
if 'timestamp' in raw_df.columns:
    print("Sample values from 'timestamp' column:")
    print(raw_df['timestamp'].sample(10).tolist())
    
    # Check unique values
    unique_timestamps = raw_df['timestamp'].nunique()
    print(f"\nNumber of unique timestamp values: {unique_timestamps}")
    
    # Check for nulls
    null_timestamps = raw_df['timestamp'].isna().sum()
    print(f"Number of null timestamp values: {null_timestamps} ({null_timestamps/len(raw_df)*100:.2f}%)")
    
    # Try different timestamp formats
    print("\nTrying different timestamp interpretations:")
    
    # First, get non-null values
    non_null_timestamps = raw_df['timestamp'].dropna()
    
    if len(non_null_timestamps) > 0:
        sample_timestamp = non_null_timestamps.iloc[0]
        
        print(f"Sample timestamp: {sample_timestamp}")
        
        # Try as Unix timestamp (seconds since epoch)
        try:
            if isinstance(sample_timestamp, (int, float)):
                unix_date = pd.to_datetime(sample_timestamp, unit='s')
                print(f"As Unix timestamp (seconds): {unix_date}")
        except Exception as e:
            print(f"Not a Unix timestamp (seconds): {e}")
            
        # Try as Unix timestamp in milliseconds
        try:
            if isinstance(sample_timestamp, (int, float)):
                unix_ms_date = pd.to_datetime(sample_timestamp, unit='ms')
                print(f"As Unix timestamp (milliseconds): {unix_ms_date}")
        except Exception as e:
            print(f"Not a Unix timestamp (milliseconds): {e}")

In [ ]:
# Check if there's another potential date source
if 'repliedAt' in raw_df.columns:
    print("Sample values from 'repliedAt' column:")
    print(raw_df['repliedAt'].sample(10).tolist())
    
    # Check for nulls
    null_replied = raw_df['repliedAt'].isna().sum()
    print(f"Number of null repliedAt values: {null_replied} ({null_replied/len(raw_df)*100:.2f}%)")
    
    # If there are non-null values, check format
    non_null_replied = raw_df['repliedAt'].dropna()
    if len(non_null_replied) > 0:
        print(f"\nFirst few non-null repliedAt values:")
        print(non_null_replied.head(5).tolist())

In [ ]:
# Attempt to parse dates from multiple sources
processed_df = raw_df.copy()

# Attempt 1: Parse the date column if it's a string
if 'date' in processed_df.columns and pd.api.types.is_object_dtype(processed_df['date']):
    try:
        print("Attempting to parse 'date' column as datetime...")
        processed_df['parsed_date_1'] = pd.to_datetime(processed_df['date'], errors='coerce')
        valid_count = processed_df['parsed_date_1'].notna().sum()
        print(f"Successfully parsed {valid_count} dates ({valid_count/len(processed_df)*100:.2f}%)")
    except Exception as e:
        print(f"Error parsing date column: {e}")

# Attempt 2: Parse the timestamp column as seconds since epoch
if 'timestamp' in processed_df.columns:
    try:
        print("\nAttempting to parse 'timestamp' column as seconds since epoch...")
        processed_df['parsed_date_2'] = pd.to_datetime(processed_df['timestamp'], unit='s', errors='coerce')
        valid_count = processed_df['parsed_date_2'].notna().sum()
        print(f"Successfully parsed {valid_count} dates ({valid_count/len(processed_df)*100:.2f}%)")
    except Exception as e:
        print(f"Error parsing timestamp column (seconds): {e}")
        
# Attempt 3: Parse the timestamp column as milliseconds since epoch
if 'timestamp' in processed_df.columns:
    try:
        print("\nAttempting to parse 'timestamp' column as milliseconds since epoch...")
        processed_df['parsed_date_3'] = pd.to_datetime(processed_df['timestamp'], unit='ms', errors='coerce')
        valid_count = processed_df['parsed_date_3'].notna().sum()
        print(f"Successfully parsed {valid_count} dates ({valid_count/len(processed_df)*100:.2f}%)")
    except Exception as e:
        print(f"Error parsing timestamp column (milliseconds): {e}")
        
# Show the results
date_cols = [col for col in processed_df.columns if col.startswith('parsed_date_')]
if date_cols:
    print("\nSample of parsed dates:")
    processed_df[date_cols].head(10)

In [ ]:
# Try custom parsers for string date formats
if 'date' in processed_df.columns and pd.api.types.is_object_dtype(processed_df['date']):
    # Get sample of non-null dates
    non_null_dates = processed_df['date'].dropna()
    
    if len(non_null_dates) > 0:
        print("Testing custom date formats on sample values...")
        sample_dates = non_null_dates.sample(min(5, len(non_null_dates))).tolist()
        
        # Common date formats to try
        formats = [
            '%Y-%m-%d',            # 2023-01-31
            '%Y/%m/%d',            # 2023/01/31
            '%d-%m-%Y',            # 31-01-2023
            '%d/%m/%Y',            # 31/01/2023
            '%m-%d-%Y',            # 01-31-2023
            '%m/%d/%Y',            # 01/31/2023
            '%Y-%m-%d %H:%M:%S',   # 2023-01-31 14:30:45
            '%d-%m-%Y %H:%M:%S',   # 31-01-2023 14:30:45
            '%m-%d-%Y %H:%M:%S',   # 01-31-2023 14:30:45
            '%b %d, %Y',           # Jan 31, 2023
            '%B %d, %Y',           # January 31, 2023
            '%d %b %Y',            # 31 Jan 2023
            '%d %B %Y',            # 31 January 2023
            '%Y%m%d'               # 20230131
        ]
        
        print(f"Sample dates: {sample_dates}")
        
        for date_format in formats:
            successful = 0
            for sample_date in sample_dates:
                if isinstance(sample_date, str):
                    try:
                        parsed = datetime.strptime(sample_date, date_format)
                        successful += 1
                    except ValueError:
                        pass
            
            if successful > 0:
                print(f"Format '{date_format}' worked for {successful}/{len(sample_dates)} samples")
                
                # Try applying this format to the whole dataset
                format_name = date_format.replace('%', '').replace(':', '').replace(' ', '_')
                col_name = f'parsed_date_{format_name}'
                
                try:
                    processed_df[col_name] = pd.to_datetime(processed_df['date'], format=date_format, errors='coerce')
                    valid_count = processed_df[col_name].notna().sum()
                    print(f"  Successfully parsed {valid_count} dates ({valid_count/len(processed_df)*100:.2f}%) using {date_format}")
                except Exception as e:
                    print(f"  Error applying format {date_format}: {e}")
        
        print("\nChecking if any custom parsers worked well:")
        custom_date_cols = [col for col in processed_df.columns if col.startswith('parsed_date_') and col not in date_cols]
        
        for col in custom_date_cols:
            valid_count = processed_df[col].notna().sum()
            if valid_count > 0:
                print(f"{col}: {valid_count} valid dates ({valid_count/len(processed_df)*100:.2f}%)")
                print(processed_df[col].head(5))

In [ ]:
# Examine some review records with version info to see if there's any correlation
if 'version' in raw_df.columns:
    print("Examining records with version info:")
    version_groups = raw_df.groupby('version').size().reset_index(name='count')
    version_groups = version_groups.sort_values('count', ascending=False)
    print(version_groups.head(10))
    
    # Check a few specific versions to see if they have date info
    for version in version_groups['version'].head(3):
        if pd.notna(version):
            print(f"\nSample records for version {version}:")
            version_sample = raw_df[raw_df['version'] == version].sample(min(3, len(raw_df[raw_df['version'] == version])))
            print(version_sample[['review_id', 'date', 'timestamp', 'version']].to_string())

In [ ]:
# Summary of findings
print("SUMMARY OF DATE DEBUGGING:")
print("==========================")

if 'date' in raw_df.columns:
    null_dates = raw_df['date'].isna().sum()
    print(f"Date column: {null_dates}/{len(raw_df)} null values ({null_dates/len(raw_df)*100:.2f}%)")
    if pd.api.types.is_object_dtype(raw_df['date']):
        print("  Data type: string/object")
    else:
        print(f"  Data type: {raw_df['date'].dtype}")

if 'timestamp' in raw_df.columns:
    null_timestamps = raw_df['timestamp'].isna().sum()
    print(f"Timestamp column: {null_timestamps}/{len(raw_df)} null values ({null_timestamps/len(raw_df)*100:.2f}%)")
    print(f"  Data type: {raw_df['timestamp'].dtype}")

# Best parsed date column
all_date_cols = [col for col in processed_df.columns if col.startswith('parsed_date_')]
if all_date_cols:
    best_col = None
    best_valid_count = 0
    
    for col in all_date_cols:
        valid_count = processed_df[col].notna().sum()
        if valid_count > best_valid_count:
            best_valid_count = valid_count
            best_col = col
    
    if best_col:
        print(f"\nBest parsed date column: {best_col}")
        print(f"  Valid dates: {best_valid_count}/{len(processed_df)} ({best_valid_count/len(processed_df)*100:.2f}%)")
        if best_valid_count > 0:
            print("  Date range:")
            print(f"    Min: {processed_df[best_col].min()}")
            print(f"    Max: {processed_df[best_col].max()}")
            
            # Sample of valid dates
            valid_dates = processed_df[processed_df[best_col].notna()]
            if len(valid_dates) > 0:
                print("\n  Sample of valid dates:")
                print(valid_dates[[best_col]].head(5))
else:
    print("\nNo successful date parsing attempts")

# Recommendation
print("\nRECOMMENDATION:")
if all_date_cols and best_valid_count > 0:
    print(f"Use the {best_col} column for date information. It successfully parsed {best_valid_count}/{len(processed_df)} dates.")
    print("Update the data loading code to use this parsing approach.")
else:
    print("The date information in the dataset appears to be missing or in an unrecognized format.")
    print("Options:")
    print("1. Continue using synthetic dates for visualization purposes")
    print("2. Investigate the data source to determine the correct date format")
    print("3. Consider using other metadata (like version numbers) as a proxy for timeframes")